In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier

from trustee import ClassificationTrustee

In [ ]:
pool_1_ips = {
    '169.231.8.190', 
    '169.231.123.195', 
    '169.231.10.199', 
    '169.231.210.93', 
    '169.231.172.165', 
}

pool_2_ips = {
    '128.111.52.37',
    '169.231.107.236',
    '169.231.111.168',
    '169.231.90.61',
}

In [ ]:
def read_dataset(prefix: str):
    dataset_1 = pd.read_csv(f'{prefix}_dataset_1.csv')
    dataset_1['Class'] = 0
    dataset_1.loc[dataset_1['Src IP'].isin(pool_1_ips), 'Class'] = 1
    ttl_data_1 = pd.read_csv(f'{prefix}_ttl_1.csv')
    dataset_1 = dataset_1.merge(ttl_data_1, on="Flow ID", how='left')
    
    dataset_2 = pd.read_csv(f'{prefix}_dataset_2.csv')
    dataset_2['Class'] = 0
    dataset_2.loc[dataset_2['Src IP'].isin(pool_2_ips), 'Class'] = 1
    ttl_data_2 = pd.read_csv(f'{prefix}_ttl_2.csv')
    dataset_2 = dataset_2.merge(ttl_data_2, on="Flow ID", how='left')
    
    dataset = pd.concat([dataset_1, dataset_2])
    dataset = dataset.replace([np.inf, -np.inf], np.nan)
    dataset = dataset.dropna(axis=0)
    dataset = dataset.drop([
        'Flow ID',
        'Src IP',
        'Dst IP',
        'Timestamp', 
        'Protocol',    # always tcp
        'Label',       # empty
    ], axis=1)
    return dataset

In [ ]:
campus_dataset = read_dataset('campus')

In [ ]:
def train_and_visualize(dataset, clf, visualize_tree = False):
    target_variable = 'Class'
    features = list(sorted(set(dataset.columns) - {target_variable}))
    x_data = dataset[features]
    y_data = dataset[target_variable]
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25)
    
    x_train = pd.DataFrame(StandardScaler().fit_transform(x_train), columns = x_train.columns)
    x_test = pd.DataFrame(StandardScaler().fit_transform(x_test), columns = x_test.columns)
    
    trained_clf = clf.fit(x_train, y_train)
    prediction = trained_clf.predict(x_test)
    print(metrics.classification_report(y_test, prediction))
    
    trustee = ClassificationTrustee(expert=trained_clf)
    trustee.fit(x_train, y_train, num_iter=10, num_stability_iter=3, samples_size=0.8)
    
    _, dt, _, score = trustee.explain()
    print(f"Training score of pruned DT: {score}")
    dt_y_pred = dt.predict(x_test)
    
    print("Model explanation global fidelity report:")
    print(metrics.classification_report(prediction, dt_y_pred))
    print("Model explanation score report:")
    print(metrics.classification_report(y_test, dt_y_pred))
    
    fig = plt.figure(figsize=(25,20))
    plot_tree(dt, feature_names=x_train.columns, class_names=['benign', 'attack'], filled=True, max_depth=3)

In [ ]:
azure_dataset = read_dataset('azure')

In [ ]:
pool_1_ips = {
    '157.245.108.149', # netunicorn-digitalocean-1
    '34.214.149.122',  # netunicorn-aws-1
}

pool_2_ips = {
    "52.43.47.231",   # netunicorn-aws-2
    "15.164.100.10",  # netunicorn-aws-3
    "170.64.144.63",  # netunicorn-digitalocean-2
}
multicloud_dataset = read_dataset('multicloud')

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
target_variable = 'Class'
features = list(sorted(set(campus_dataset.columns) - {target_variable}))
x_train = campus_dataset[features]
y_train = campus_dataset[target_variable]
x_test = azure_dataset[features]
y_test = azure_dataset[target_variable]
x_test_2 = multicloud_dataset[features]
y_test_2 = multicloud_dataset[target_variable]

In [ ]:
x_train = pd.DataFrame(StandardScaler().fit_transform(x_train), columns = x_train.columns)
x_test = pd.DataFrame(StandardScaler().fit_transform(x_test), columns = x_test.columns)
x_test_2 = pd.DataFrame(StandardScaler().fit_transform(x_test_2), columns = x_test_2.columns)

In [ ]:
classifiers = [
    # MLPClassifier(),
    # GradientBoostingClassifier(),
    RandomForestClassifier(max_depth=3)
]
for clf in classifiers:
    print(clf)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_train)
    print("campus dataset training accuracy: ")
    print(metrics.classification_report(y_train, y_pred))

    print("Azure dataset test accuracy: ")
    y_pred = clf.predict(x_test)
    print(metrics.classification_report(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))
    
    print("Multicloud dataset test accuracy: ")
    y_pred = clf.predict(x_test_2)
    print(metrics.classification_report(y_test_2, y_pred))
    print(metrics.confusion_matrix(y_test_2, y_pred))
    print('#' * 10 + '\n')

In [ ]:
clf = RandomForestClassifier()
train_and_visualize(campus_dataset, clf)
print("Azure dataset test accuracy: ")
y_pred = clf.predict(azure_dataset[features])
print(metrics.classification_report(azure_dataset[target_variable], y_pred))
print(metrics.confusion_matrix(azure_dataset[target_variable], y_pred))

In [ ]:
clf = DecisionTreeClassifier(max_depth=2)
clf.fit(x_train, y_train)

In [ ]:
y_pred = clf.predict(x_test)
print(metrics.classification_report(azure_dataset[target_variable], y_pred))
print(metrics.confusion_matrix(azure_dataset[target_variable], y_pred))

fig = plt.figure(figsize=(25,20))
plot_tree(clf, feature_names=x_train.columns, class_names=['benign', 'attack'], filled=True, max_depth=2)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(x_test, y_test)

In [ ]:
fig = plt.figure(figsize=(25,20))
plot_tree(clf, feature_names=x_train.columns, class_names=['benign', 'attack'], filled=True, max_depth=2)

## Adding OOD example

In [ ]:
def read_ood_dataset(prefix: str):
    dataset = pd.read_csv(f'{prefix}_dataset.csv')
    dataset['Class'] = 0
    dataset.loc[dataset['Src IP'].isin(pool_ips), 'Class'] = 1
    ttl_data = pd.read_csv(f'{prefix}_ttl.csv')
    dataset = dataset.merge(ttl_data, on="Flow ID", how='left')
    dataset = dataset.replace([np.inf, -np.inf], np.nan)
    dataset = dataset.dropna(axis=0)
    dataset = dataset.drop([
        'Flow ID',
        'Src IP',
        'Dst IP',
        'Timestamp', 
        'Protocol',    # always tcp
        'Label',       # empty
    ], axis=1)
    return dataset

In [ ]:
pool_ips = {
    '169.231.10.199',
    '169.231.210.93',
    '169.231.8.190',
    '169.231.123.195',
}
hydra_dataset = read_ood_dataset("ood")

In [ ]:
x_hydra_test = hydra_dataset[features]
y_hydra_test = hydra_dataset[target_variable]
x_hydra_test = pd.DataFrame(StandardScaler().fit_transform(x_hydra_test), columns = x_hydra_test.columns)

for clf in classifiers:
    print(clf)
    print("OOD test dataset accuracy: ")
    y_pred = clf.predict(x_hydra_test)
    print(metrics.classification_report(y_hydra_test, y_pred))
    print(metrics.confusion_matrix(y_hydra_test, y_pred))

## Dropping features test

In [ ]:
x_train = x_train.drop(['TTL', 'Bwd Init Win Bytes'], axis=1)
x_test = x_test.drop(['TTL', 'Bwd Init Win Bytes'], axis=1)
x_test_2 = x_test_2.drop(['TTL', 'Bwd Init Win Bytes'], axis=1)

In [ ]:
classifiers = [
    MLPClassifier(),
    GradientBoostingClassifier(),
    RandomForestClassifier(max_depth=3)
]
for clf in classifiers:
    print(clf)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_train)
    print("campus dataset training accuracy: ")
    print(metrics.classification_report(y_train, y_pred))

    print("Azure dataset test accuracy: ")
    y_pred = clf.predict(x_test)
    print(metrics.classification_report(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))
    
    print("Multicloud dataset test accuracy: ")
    y_pred = clf.predict(x_test_2)
    print(metrics.classification_report(y_test_2, y_pred))
    print(metrics.confusion_matrix(y_test_2, y_pred))
    print('#' * 10 + '\n')